In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [3]:
tickers = {
    "^IRX": "3-Month",  # 13 Week T-Bill (proxy for 3-month)
    "^FVX": "5-Year",   # 5-Year T-Note
    "^TNX": "10-Year",  # 10-Year T-Note
    "^TYX": "30-Year",  # 30-Year T-Bond
}

# Step 2: Fetch data for the past 6 months
data = {}
for ticker, label in tickers.items():
    try:
        # Download the data and extract adjusted close prices
        df = yf.download(ticker, start="2024-07-01", end="2025-01-01")
        if not df.empty:  # Ensure the data is valid
            data[label] = df["Close"]
        else:
            print(f"No data available for {label} ({ticker})")
    except Exception as e:
        print(f"Failed to download data for {label} ({ticker}): {e}")


#create a new dataframe called yield data and set the index to the date column
mat = 30
yield_data = pd.DataFrame(columns=['Date','Maturity','Yield'])
for k,v in data.items():
    if k == "3-Month":
        mat = 3
    elif k == "5-Year":
        mat = 60
    elif k == "10-Year":
        mat = 120
    elif k == "30-Year":
        mat = 360
    for _,row in v.iterrows():
        df2 = pd.DataFrame([[row.name,mat,row[0]]], columns=['Date','Maturity','Yield'])
        yield_data = pd.concat([yield_data, df2])

yield_data.set_index('Date', inplace=True)
yield_data.index = pd.to_datetime(yield_data.index)
yield_data

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/var/folders/4r/dygtxtn155l8zwg6_s8jtpw80000gn/T/ipykernel_33460/460736673.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df2 = pd.DataFrame([[row.name,mat,row[0]]], columns=['Date','Maturity','Yield'])
/var/folders/4r/dygtxtn155l8zwg6_s8jtpw80000gn/T/ipykernel_33460/460736673.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  yield_data = pd.c

,Maturity,Yield
Date,,
2024-07-01,3,5.218
2024-07-02,3,5.228
2024-07-03,3,5.230
2024-07-05,3,5.225
2024-07-08,3,5.218
...,...,...
2024-12-24,360,4.765
2024-12-26,360,4.762
2024-12-27,360,4.811


In [3]:
# Step 3: Prepare data for 3D plotting
# Create a grid for maturities (x), time (y), and interest rates (z)
maturities = [3, 60, 120, 360]  # Maturities in months
dates = yield_data.index
x, y = np.meshgrid(maturities, dates)
z = yield_data.to_numpy()

In [4]:
# Pivot the yield_data DataFrame for easier plotting
pivot_yield_data = yield_data.reset_index().pivot(index='Date', columns='Maturity', values='Yield')

# Ensure maturities are in the correct order
pivot_yield_data = pivot_yield_data[[3, 60, 120, 360]]  # Align with maturities in months

# Extract the z-axis (yield values), x-axis (maturities), and y-axis (dates)
z = pivot_yield_data.to_numpy()  # Yield values
x = np.array([3, 60, 120, 360])  # Maturities in months
y = pivot_yield_data.index  # Dates

# Create the 3D grid for plotting
x_grid, y_grid = np.meshgrid(x, y)

# Generate the 3D plot
fig = go.Figure(data=[go.Surface(
    z=z,  # Use the direct yield matrix
    x=x_grid,  # Maturities
    y=y_grid,  # Dates
    colorscale="Viridis",
    colorbar=dict(title="Yield (%)"),
)])

# Update layout
fig.update_layout(
    title="3D Interest Rate Term Structure",
    scene=dict(
        xaxis=dict(title="Maturity (Months)"),
        yaxis=dict(title="Date"),
        zaxis=dict(title="Yield (%)"),
    ),
    margin=dict(l=0, r=0, b=0, t=50),
)

# Display the plot
fig.show()


KeyError: "None of [Index([3, 60, 120, 360], dtype='int64', name='Maturity')] are in the [columns]"